# Data Warehouse Truven QA - Claim Detail

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [20]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [21]:
connection = psycopg2.connect(get_dsn()+' keepalives=1 keepalives_idle=30 keepalives_interval=10')
connection.autocommit = True

In [22]:
year_df = pd.read_sql('select distinct year from dw_staging.trum_claim_detail;', con=connection)
tables = ['ccaes', 'mdcrs', 'mdcro', 'ccaeo']

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_detail table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the truven schema. 


In [32]:
query = ''' drop table if exists qa_reporting.dw_truv_claim_detail_counts;
create table qa_reporting.dw_truv_claim_detail_counts
(
    data_source bpchar(4),
    calendar_year int,
    table_src text,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_diff_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [34]:
with connection.cursor() as cursor:
    for year in tqdm(year_df['year']):
        query = f'''
        insert into qa_reporting.dw_truv_claim_detail_counts
        (data_source, calendar_year, table_src, dw_row_count, date_generated)
        select data_source, year, table_id_src, count(*), current_date
        from dw_staging.truc_claim_detail
        where year = {year}
        group by 1, 2, 3
        '''
        
        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_uth_clm_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct uth_claim_id) as count 
            from dw_staging.truc_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_src_clm_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct claim_id_src) as count 
            from dw_staging.truc_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_uth_mbr_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct uth_member_id) as count 
            from dw_staging.truc_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)
        
        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_src_mbr_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct member_id_src) as count 
            from dw_staging.truc_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

100%|██████████| 12/12 [2:28:45<00:00, 743.77s/it] 


In [33]:
with connection.cursor() as cursor:
    for year in tqdm(year_df['year']):
        query = f'''
        insert into qa_reporting.dw_truv_claim_detail_counts
        (data_source, calendar_year, table_src, dw_row_count, date_generated)
        select data_source, year, table_id_src, count(*), current_date
        from dw_staging.trum_claim_detail
        where year = {year}
        group by 1, 2, 3
        '''
        
        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_uth_clm_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct uth_claim_id) as count 
            from dw_staging.trum_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_src_clm_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct claim_id_src) as count 
            from dw_staging.trum_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_uth_mbr_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct uth_member_id) as count 
            from dw_staging.trum_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)
        
        query = f'''
        update qa_reporting.dw_truv_claim_detail_counts b
        set dw_src_mbr_id_count = count
        from (
            select data_source, year, table_id_src,   count(distinct member_id_src) as count 
            from dw_staging.trum_claim_detail
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

100%|██████████| 12/12 [40:41<00:00, 203.45s/it]


In [36]:
with connection.cursor() as cursor:
    query = '''update qa_reporting.dw_truv_claim_detail_counts a
    set src_row_count = b.row_count,
    row_count_diff = dw_row_count - b.row_count,
    row_count_diff_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
    src_clm_count = clm_count,
    clm_count_diff = dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from qa_reporting.truven_counts b
    where a.calendar_year = b.year
    and a.table_src = b.table_name
    ;
    '''

    cursor.execute(query)

In [37]:
query = '''select * from qa_reporting.dw_truv_claim_detail_counts;'''

df = pd.read_sql(query, con=connection)
df.sort_values('calendar_year')

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
40,trum,2011,mdcrs,32522421,32535953,-13532,0.041591,10833438,10831969,10836750,-3312,0.030563,794174,794174,794571,-397,0.049964,2023-07-26
38,trum,2011,mdcro,234159725,234255047,-95322,0.040692,106545093,106542800,106577308,-32215,0.030227,4743651,4743651,4750926,-7275,0.153128,2023-07-26
16,truc,2011,ccaes,81538719,81640170,-101451,0.124266,22335352,22332253,22342714,-7362,0.032950,2347911,2347911,2348969,-1058,0.045041,2023-07-26
14,truc,2011,ccaeo,1076951967,1077661934,-709967,0.065880,465618812,465615399,465722367,-103555,0.022235,42488517,42488517,42511345,-22828,0.053699,2023-07-26
17,truc,2012,ccaes,82150319,82224989,-74670,0.090812,21970001,21965821,21970624,-623,0.002836,2288983,2288983,2289517,-534,0.023324,2023-07-26
41,trum,2012,mdcrs,32194159,32200251,-6092,0.018919,10013948,10011917,10014407,-459,0.004583,714025,714025,714226,-201,0.028142,2023-07-26
39,trum,2012,mdcro,229684913,229768218,-83305,0.036256,99423240,99421263,99450749,-27509,0.027661,4384491,4384491,4391031,-6540,0.148940,2023-07-26
15,truc,2012,ccaeo,1121364740,1121961205,-596465,0.053163,475864303,475860103,475937265,-72962,0.015330,43278501,43278501,43288123,-9622,0.022228,2023-07-26
12,truc,2013,ccaeo,875128399,875696525,-568126,0.064877,374376302,374372110,374452293,-75991,0.020294,33649435,33649435,33659811,-10376,0.030826,2023-07-26
36,trum,2013,mdcro,207987235,208095580,-108345,0.052065,88020932,88019152,88058549,-37617,0.042718,3805830,3805830,3812531,-6701,0.175763,2023-07-26


In [38]:
df.sort_values('row_count_diff_percentage', ascending=False)[['calendar_year', 'table_src', 'row_count_diff_percentage', 'clm_count_percentage', 'mbr_count_percentage']]

,calendar_year,table_src,row_count_diff_percentage,clm_count_percentage,mbr_count_percentage
19,2022,ccaes,0.499326,0.383641,0.856400
23,2017,ccaes,0.262790,0.059769,0.112192
21,2016,ccaes,0.233203,0.057319,0.106096
10,2015,ccaes,0.185213,0.012971,0.048973
4,2018,ccaes,0.182036,0.000920,0.085264
22,2017,ccaeo,0.158912,0.069860,0.072138
3,2020,ccaes,0.155065,0.016350,0.057544
7,2019,ccaes,0.150225,0.013265,0.047117
20,2016,ccaeo,0.144871,0.065753,0.068810
18,2022,ccaeo,0.137395,0.041223,0.076549


## Place of Service

In [40]:
query = '''drop table if exists qa_reporting.dw_truv_pos;
select data_source, year, place_of_service, count(*)
into qa_reporting.dw_truv_pos
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_pos
select data_source, year, place_of_service, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [41]:
pos_df = pd.read_sql('select * from qa_reporting.dw_truv_pos;', con=connection)
pos_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,place_of_service,count
0,trum,2011,11,110831435
1,trum,2016,81,5209247
2,trum,2016,57,16892
3,truc,2012,42,17356
4,trum,2014,35,13
...,...,...,...,...
1078,trum,2017,62,383371
1079,trum,2011,20,250687
1080,truc,2017,1.,103970
1081,truc,2022,34,103321


In [42]:
pos_df['place_of_service'].unique()

array(['11', '81', '57', '42', '35', '3', '22', '33', '1', '1.', '51',
       '26', '53', '24', '71', '23', '56', '28', '25', '60', '34', '10',
       '32', '41', '95', '72', '12', '14', '3.', '2', '50', '31', None,
       '13', '20', '15', '65', '18', '16', '49', '52', '55', '19', '21',
       '2.', '54', '17', '62', '99', '61', '58', '4.', '4', '27'],
      dtype=object)

In [50]:
query = '''
select a.* 
from qa_reporting.dw_truv_pos a
left join reference_tables.ref_place_of_service b
on a.place_of_service = b.place_of_treatment_cd
where b.place_of_treatment_cd is null
and a.place_of_service is not null
;
'''
pos_df = pd.read_sql(query, con=connection)
pos_df.sort_values(['year', 'place_of_service'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,place_of_service,count
40,truc,2011,1.,235271
44,trum,2011,1.,182179
57,truc,2011,3.,38641
58,trum,2011,3.,1137
4,truc,2011,4.,287
...,...,...,...,...
26,trum,2022,2,195042
69,trum,2022,3,701
75,truc,2022,3,128647
29,truc,2022,4,473


In [51]:
pos_df.groupby(['data_source', 'year']).sum()

count
data_source year          
truc        2011    274199
            2012    373603
            2013    241795
            2014    353861
            2015    234087
            2016    253920
            2017    201004
            2018    289314
            2019    524434
            2020  11384116
            2021  13606349
            2022   8510923
trum        2011    183690
            2012    194857
            2013    169902
            2014    159548
            2015     27232
            2016     25101
            2017     12418
            2018     16114
            2019     19818
            2020    392950
            2021    269405
            2022    328456

In [44]:
pd.read_sql('select distinct stdplac from truven.mdcrs;', con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,stdplac
0,NaN
1,52.0
2,12.0
3,1.0
4,49.0
5,19.0
6,22.0
7,24.0
8,32.0
9,11.0


In [45]:
pd.read_sql('select distinct stdplac from truven.mdcro;', con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,stdplac
0,NaN
1,23.0
2,12.0
3,54.0
4,21.0
5,51.0
6,52.0
7,1.0
8,49.0
9,13.0


## Revenue Code

Looking at how many claims have invalid revenue codes.

In [53]:
query = '''drop table if exists qa_reporting.dw_truv_revenue_cd;
select data_source, year, revenue_cd, count(*)
into qa_reporting.dw_truv_revenue_cd
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_pos
select data_source, year, revenue_cd, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [54]:
query = '''
select a.* 
from qa_reporting.dw_truv_revenue_cd a
left join reference_tables.ref_revenue_code b
on a.revenue_cd = b.revenue_cd
where b.revenue_cd is null
and a.revenue_cd is not null
;
'''
invalid_rev_cd_df = pd.read_sql(query, con=connection)
invalid_rev_cd_df.sort_values(['year', 'revenue_cd'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,revenue_cd,count
596,trum,2011,0002,15
651,trum,2011,0003,2
859,trum,2011,0004,2
240,trum,2011,0005,1
71,trum,2011,0006,1
...,...,...,...,...
466,trum,2022,4507,1
761,trum,2022,4636,1
568,trum,2022,9213,1
542,trum,2022,9900,1


In [55]:
pd.DataFrame(invalid_rev_cd_df['revenue_cd'].unique())

,0
0,0050
1,9035
2,0195
3,0090
4,1212
...,...
529,0025
530,0227
531,0638
532,3971


In [56]:
invalid_rev_cd_sum = invalid_rev_cd_df.groupby('year').sum()
invalid_rev_cd_sum

,count
year,
2011,66442
2012,69576
2013,68633
2014,66143
2015,44159
2016,40535
2017,25206
2018,14807
2019,26127


In [57]:
rev_cd_df = pd.read_sql('select data_source, year, sum(count) from qa_reporting.dw_truv_revenue_cd group by 1,2;', con=connection)
rev_cd_df['sum'] = rev_cd_df['sum'].astype(int)
rev_cd_df = rev_cd_df.set_index('year')
rev_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,sum
year,,
2012,trum,261879072
2019,trum,112172702
2020,trum,106215660
2016,trum,134338049
2015,trum,138140504
2022,trum,88608557
2014,trum,219788855
2011,trum,266682146
2017,trum,92328976


We should have a very low percentage of claim lines that have revenue codes with invalid revenue codes. If the percent if > 1%, need to investigate further

In [58]:
100. * invalid_rev_cd_sum['count'] / rev_cd_df['sum']

year
2011    0.024914
2012    0.026568
2013    0.028955
2014    0.030094
2015    0.031967
2016    0.030174
2017    0.027300
2018    0.022376
2019    0.023292
2020    0.012431
2021    0.018054
2022    0.015726
dtype: float64

## Bill Type Code

Looking at how many claims have invalid bill type codes. This occurs when the claim has a 2 character bill type code in the raw data.

In [59]:
query = '''drop table if exists qa_reporting.dw_truv_bill_cd;
select data_source, year, bill, count(*)
into qa_reporting.dw_truv_bill_cd
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_bill_cd
select data_source, year, bill, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [62]:
query = '''
select a.* 
from qa_reporting.dw_truv_bill_cd a
left join reference_tables.ref_bill_type_cd b
on a.bill = b.bill_type_cd
where b.bill_type_cd is null
and a.bill is not null
;
'''
invalid_bill_cd_df = pd.read_sql(query, con=connection)
invalid_bill_cd_df.sort_values(['year', 'bill'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,bill,count
259,truc,2011,11,512346
268,trum,2011,11,4824
459,truc,2011,12,3
466,trum,2011,12,31
476,truc,2011,13,2463152
...,...,...,...,...
120,truc,2022,82,51
58,truc,2022,83,1435
514,truc,2022,85,729
373,truc,2022,86,1310


In [63]:
pd.DataFrame(invalid_bill_cd_df['bill'].unique())

,0
0,64
1,77
2,53
3,92
4,56
...,...
57,21
58,54
59,31
60,19


In [64]:
invalid_bill_cd_sum = invalid_bill_cd_df.groupby('year').sum()
invalid_bill_cd_sum

,count
year,
2011,3256738
2012,3236522
2013,3809923
2014,4471311
2015,2744882
2016,216205
2017,219078
2018,218208
2019,1515117


In [65]:
bill_cd_df = pd.read_sql('select year, sum(count) from qa_reporting.dw_truv_bill_cd group by 1', con=connection)
bill_cd_df['sum'] = bill_cd_df['sum'].astype(int)
bill_cd_df = bill_cd_df.set_index('year')
bill_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sum
year,
2018,693631518
2020,625018402
2021,663758055
2014,1238422884
2019,714793930
2011,1425172832
2012,1465394131
2022,480470842
2017,702718911


We should have a very low percentage of claim lines that have bill type codes with invalid bill type codes. If the percent if > 1%, need to investigate further

In [66]:
100. * invalid_bill_cd_sum['count'] / bill_cd_df['sum']

year
2011    0.228515
2012    0.220864
2013    0.324327
2014    0.361049
2015    0.351712
2016    0.027009
2017    0.031176
2018    0.031459
2019    0.211966
2020    0.194926
2021    0.236925
2022    0.242183
dtype: float64

## CPT HCPCS Code

In [67]:
query = '''drop table if exists qa_reporting.dw_truv_cpt_proc_counts;
select data_source, year, cpt_hcpcs_cd, count(*) as proc_count
into qa_reporting.dw_truv_cpt_proc_counts
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_cpt_proc_counts
select data_source, year, cpt_hcpcs_cd, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [68]:
cpt_proc_cd_df = pd.read_sql('select * from qa_reporting.dw_truv_cpt_proc_counts;', con=connection)
cpt_proc_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,cpt_hcpcs_cd,proc_count
0,trum,2018,A4362,3027
1,trum,2019,29085,387
2,trum,2011,12006,261
3,trum,2018,22852,199
4,trum,2018,36245,2704
...,...,...,...,...
348536,truc,2013,D2542,5
348537,truc,2022,86833,8687
348538,truc,2021,90718,3
348539,truc,2013,96360,336804


Currently it may be hard to determine how many invalid CPT and HCPCS there in the data due to the list of codes we have for as a reference. There are two reference tables that can be used, **reference_tables.mrconso_cpt_hcpcs_hcpt** and **reference_tables.cpt_hcpc**

In [69]:
query = '''
select a.*
from qa_reporting.dw_truv_cpt_proc_counts a
left join (select distinct code from reference_tables.mrconso_cpt_hcpcs_hcpt) b
on a.cpt_hcpcs_cd = b.code
where b.code is null
;
'''

invalid_proc_df = pd.read_sql(query, con=connection)
invalid_proc_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,cpt_hcpcs_cd,proc_count
0,truc,2016,C9349,362
1,truc,2022,86849,1275
2,truc,2016,90646,19
3,truc,2012,V2117,4
4,truc,2014,90813,3
...,...,...,...,...
35111,trum,2011,C9246,1
35112,truc,2011,L7274,120
35113,trum,2011,22523,6043
35114,trum,2021,D0274,23


In [70]:
invalid_proc_df.groupby('year')['proc_count'].sum()

year
2011    265335289
2012    275497380
2013    201180509
2014    197600323
2015    120429170
2016    112858401
2017     85079444
2018     68188181
2019     65933063
2020     53026012
2021     52510960
2022     37990726
Name: proc_count, dtype: int64

In [71]:
proc_comp_df = pd.DataFrame({'overall_proc_count': cpt_proc_cd_df.groupby('year')['proc_count'].sum(),
                            'invalid_proc_count': invalid_proc_df.groupby('year')['proc_count'].sum(),
                            'valid_proc_count': cpt_proc_cd_df.groupby('year')['proc_count'].sum() - invalid_proc_df.groupby('year')['proc_count'].sum()})
proc_comp_df.loc[proc_comp_df['valid_proc_count'].isna(),'valid_proc_count'] = proc_comp_df.loc[proc_comp_df['valid_proc_count'].isna(),'overall_proc_count']
proc_comp_df['valid_proc_count'] =  proc_comp_df['valid_proc_count'].astype(int)
proc_comp_df['invalid_to_valid_percent'] = 100. * proc_comp_df['invalid_proc_count'] / proc_comp_df['valid_proc_count']
proc_comp_df  

,overall_proc_count,invalid_proc_count,valid_proc_count,invalid_to_valid_percent
year,,,,
2011,1425172832,265335289,1159837543,22.876936
2012,1465394131,275497380,1189896751,23.153049
2013,1174715607,201180509,973535098,20.664947
2014,1238422884,197600323,1040822561,18.985015
2015,780433995,120429170,660004825,18.246711
2016,800493749,112858401,687635348,16.412536
2017,702718911,85079444,617639467,13.774936
2018,693631518,68188181,625443337,10.902375
2019,714793930,65933063,648860867,10.161356


## Discharge Status

Checking if there are invalid discharge status codes.

Ignore lines with a NULL discharge status code. Most lines have no discharge status possibly due to raw data not providing this or other reasons.

In [72]:
query = '''drop table if exists qa_reporting.dw_truv_discharge_counts;
select data_source, year, discharge_status, count(*) as proc_count
into qa_reporting.dw_truv_discharge_counts
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_discharge_counts
select data_source, year, discharge_status, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [73]:
dschrg_cd_df = pd.read_sql('select * from qa_reporting.dw_truv_discharge_counts;', con=connection)
dschrg_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,discharge_status,proc_count
0,trum,2019,07,28197
1,truc,2022,None,372692735
2,truc,2020,70,55621
3,trum,2017,86,1659
4,trum,2014,20,1150165
...,...,...,...,...
905,trum,2021,99,1
906,trum,2017,81,1061
907,truc,2017,03,855464
908,truc,2020,None,487622824


In [74]:
dschrg_cd_df['discharge_status'].unique()

array(['07', None, '70', '86', '20', '08', '64', '95', '01', '65', '91',
       '04', '53', '05', '83', '09', '99', '81', '63', '90', '51', '88',
       '06', '92', '69', '03', '98', '71', '42', '50', '54', '40', '61',
       '94', '93', '30', '21', '41', '89', '02', '62', '85', '84', '66',
       '82', '87', '60', '72', '43', '80'], dtype=object)

In [75]:
dschrg_cd_df[dschrg_cd_df['discharge_status'].isna()]

,data_source,year,discharge_status,proc_count
1,truc,2022,None,372692735
6,trum,2019,None,100383512
7,trum,2021,None,86837896
239,truc,2012,None,1125540462
361,trum,2016,None,119918298
408,truc,2013,None,878644054
441,truc,2021,None,536961989
454,trum,2020,None,95038028
463,truc,2017,None,576257738
468,trum,2018,None,59792998


In [76]:
dschrg_cd_df = dschrg_cd_df[~dschrg_cd_df['discharge_status'].isna()]
dschrg_cd_df

,data_source,year,discharge_status,proc_count
0,trum,2019,07,28197
2,truc,2020,70,55621
3,trum,2017,86,1659
4,trum,2014,20,1150165
5,truc,2012,08,961
...,...,...,...,...
904,truc,2015,92,388
905,trum,2021,99,1
906,trum,2017,81,1061
907,truc,2017,03,855464


In [77]:
query = '''
select a.*
from qa_reporting.dw_truv_discharge_counts a
left join reference_tables.ref_discharge_status b
on a.discharge_status = b.discharge_status
where b.discharge_status is null
and a.discharge_status is not null
;
'''

invalid_dschrg_df = pd.read_sql(query, con=connection)
invalid_dschrg_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,discharge_status,proc_count
0,trum,2016,54,138
1,truc,2019,54,47
2,truc,2014,99,512
3,trum,2021,54,195
4,trum,2020,54,330
5,truc,2017,54,42
6,truc,2016,98,238
7,trum,2021,53,19
8,trum,2014,99,71
9,truc,2014,54,19


In [78]:
dschrg_comp_df = pd.DataFrame({'overall_proc_count': dschrg_cd_df.groupby('year')['proc_count'].sum(),
                            'invalid_proc_count': invalid_dschrg_df.groupby('year')['proc_count'].sum(),
                            'valid_proc_count': dschrg_cd_df.groupby('year')['proc_count'].sum() - invalid_dschrg_df.groupby('year')['proc_count'].sum()})
dschrg_comp_df.loc[dschrg_comp_df['valid_proc_count'].isna(),'valid_proc_count'] = dschrg_comp_df.loc[dschrg_comp_df['valid_proc_count'].isna(),'overall_proc_count']
dschrg_comp_df['valid_proc_count'] =  dschrg_comp_df['valid_proc_count'].astype(int)
dschrg_comp_df['invalid_to_valid_percent'] = 100. * dschrg_comp_df['invalid_proc_count'] / dschrg_comp_df['valid_proc_count']
dschrg_comp_df 

,overall_proc_count,invalid_proc_count,valid_proc_count,invalid_to_valid_percent
year,,,,
2011,109127326,NaN,109127326,NaN
2012,108887838,NaN,108887838,NaN
2013,87417375,385.0,87416990,0.000440
2014,89561039,2114.0,89558925,0.002360
2015,56667138,4726.0,56662412,0.008341
2016,53974459,7954.0,53966505,0.014739
2017,43911979,8939.0,43903040,0.020361
2018,41458793,1875.0,41456918,0.004523
2019,44785289,2631.0,44782658,0.005875


## DRG CD

Checking DRG values in this column. There is no reference table to match the DRG values used in Truven so cannot validate DRG codes in data.

In [80]:
query = '''drop table if exists qa_reporting.dw_truv_drg_counts;
select data_source, year, drg_cd, count(*) as drg_count
into qa_reporting.dw_truv_drg_counts
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_drg_counts
select data_source, year, drg_cd, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [81]:
drg_df = pd.read_sql('select * from qa_reporting.dw_truv_drg_counts;', con=connection)
drg_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,drg_cd,drg_count
0,trum,2016,021,680
1,trum,2014,669,39931
2,trum,2019,533,1082
3,trum,2013,265,2626
4,trum,2019,416,493
...,...,...,...,...
17861,truc,2022,876,26300
17862,truc,2022,795,352410
17863,truc,2018,958,52379
17864,truc,2018,198,1248


In [82]:
drg_df['drg_cd'].unique()

array(['021', '669', '533', '265', '416', '316', '735', '133', '204',
       '871', '342', '576', '950', '756', '985', '562', '882', '137',
       '684', '847', '022', '940', '565', '505', '916', '605', '502',
       '068', '537', '223', '467', '559', '475', '266', '233', '699',
       '660', '470', '205', '092', '125', '182', '281', '130', '963',
       '113', '236', '273', '019', '218', '886', '082', '488', '765',
       '330', '662', '456', '432', '168', '918', '024', '436', '623',
       '853', '180', '693', '615', '335', '096', '158', '259', '846',
       '713', '071', '352', '331', '761', '086', '822', '849', '222',
       '291', '489', '095', '806', '687', '737', '710', '264', '579',
       '816', '067', '203', '004', '959', '376', '197', '314', '444',
       '517', '468', '164', '405', '247', '380', '536', '683', '157',
       '454', '514', '904', '099', '299', '494', '539', '880', '061',
       '509', '358', '340', '598', '206', '138', '603', '711', '304',
       '825', '869',

In [83]:
drg_df[drg_df['drg_cd'].isna()]

,data_source,year,drg_cd,drg_count
931,truc,2021,None,534228204
1218,truc,2016,None,622737541
2953,truc,2015,None,599156710
3299,trum,2018,None,59179228
3939,truc,2014,None,951444079
4033,trum,2019,None,99558720
4412,trum,2011,None,234159725
4775,trum,2021,None,85909861
4807,truc,2017,None,572310742
4977,truc,2020,None,485095032


In [89]:
drg_df

,data_source,year,drg_cd,drg_count
0,trum,2016,021,680
1,trum,2014,669,39931
2,trum,2019,533,1082
3,trum,2013,265,2626
4,trum,2019,416,493
...,...,...,...,...
17861,truc,2022,876,26300
17862,truc,2022,795,352410
17863,truc,2018,958,52379
17864,truc,2018,198,1248


In [120]:
comp = pd.merge(left=drg_df, right=df.loc[:, ['data_source', 'calendar_year', 'table_src', 'dw_row_count', 'dw_uth_clm_id_count']].groupby(['data_source', 'calendar_year']).sum(),
                left_on=['year', 'data_source'], right_on=['calendar_year', 'data_source'], how='outer')
comp = comp[['data_source', 'year', 'drg_cd', 'drg_count', 'dw_row_count', 'dw_uth_clm_id_count']]
comp.sort_values('drg_count')

,data_source,year,drg_cd,drg_count,dw_row_count,dw_uth_clm_id_count
5528,trum,2012,970,1,261879072,109437188
8105,truc,2015,295,1,642293491,266301958
5547,trum,2012,790,2,261879072,109437188
5995,trum,2020,768,2,106215660,44287269
16682,truc,2017,218,2,610389935,255843475
...,...,...,...,...,...,...
13404,truc,2016,None,622737541,666155700,275643201
14750,truc,2013,None,875128399,937686122,391186544
10478,truc,2014,None,951444079,1018634029,417694944
12590,truc,2011,None,1076951967,1158490686,487954164


Many claims do not have DRG codes meaning that there will be a low percentage of drg to claim ratios. 

In [122]:
drg_sum_df = comp[~comp['drg_cd'].isna()].groupby(['data_source', 'year', 'dw_uth_clm_id_count'])['drg_count'].sum()
drg_sum_df = drg_sum_df.reset_index()
drg_sum_df['type_to_id'] = 1. * (drg_sum_df['drg_count'] / drg_sum_df['dw_uth_clm_id_count'])
drg_sum_df

,data_source,year,dw_uth_clm_id_count,drg_count,type_to_id
0,truc,2011,487954164,81538719,0.167103
1,truc,2012,497834304,82150319,0.165015
2,truc,2013,391186544,62557723,0.159918
3,truc,2014,417694944,67189950,0.160859
4,truc,2015,266301958,43136781,0.161984
5,truc,2016,275643201,43418159,0.157516
6,truc,2017,255843475,38079193,0.148838
7,truc,2018,262330645,38913161,0.148336
8,truc,2019,252953189,36992224,0.146241
9,truc,2020,227363165,33707710,0.148255


## Provider Type

In [94]:
query = '''drop table if exists qa_reporting.dw_truv_provider_type_counts;
select data_source, year, provider_type, count(*) as type_count
into qa_reporting.dw_truv_provider_type_counts
from dw_staging.trum_claim_detail
group by 1,2,3
;

insert into qa_reporting.dw_truv_provider_type_counts
select data_source, year, provider_type, count(*)
from dw_staging.truc_claim_detail
group by 1,2,3
;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [95]:
pt_df = pd.read_sql('select * from qa_reporting.dw_truv_provider_type_counts;', con=connection)
pt_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,provider_type,type_count
0,trum,2020,825,1511997
1,trum,2012,453,746
2,trum,2017,150,624304
3,trum,2012,25,490012
4,trum,2020,350,600889
...,...,...,...,...
3169,truc,2014,208,125162
3170,truc,2015,380,3189590
3171,truc,2016,23,49453
3172,truc,2019,140,1180303


In [106]:
pt_df[pt_df['provider_type'] == '1']

,data_source,year,provider_type,type_count
13,truc,2022,1,76263632
330,trum,2022,1,22445434
528,trum,2018,1,17472888
570,truc,2020,1,106757359
642,trum,2020,1,25221032
648,truc,2014,1,230622362
682,truc,2013,1,201064925
871,trum,2011,1,60217982
946,trum,2014,1,60436900
989,truc,2021,1,113575309


All the provider type values in DW matches with the values Truven uses. There are several rows with no provider type.

In [96]:
query = '''
select a.*
from qa_reporting.dw_truv_provider_type_counts a
left join reference_tables.truven_prov_specialty_cds b
on a.provider_type = b.specialty_cd
where b.specialty_cd is null
;
'''

invalid_pt_df = pd.read_sql(query, con=connection)
invalid_pt_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,year,provider_type,type_count
0,truc,2017,None,15009933
1,truc,2014,None,15728059
2,trum,2011,None,12004686
3,trum,2020,None,1901654
4,truc,2011,None,72936729
5,truc,2021,None,12605665
6,trum,2018,None,2705650
7,truc,2019,None,13700336
8,trum,2015,None,4482415
9,trum,2016,None,4273716


In [116]:
comp = pd.merge(left=pt_df, right=df.loc[:, ['data_source', 'calendar_year', 'dw_row_count', 'dw_uth_clm_id_count']].groupby(['data_source', 'calendar_year']).sum(),
                left_on=['year', 'data_source'], right_on=['calendar_year', 'data_source'], how='outer')
comp = comp[['data_source', 'year', 'provider_type', 'type_count', 'dw_row_count', 'dw_uth_clm_id_count']]
comp.sort_values('type_count')

,data_source,year,provider_type,type_count,dw_row_count,dw_uth_clm_id_count
3003,trum,2013,41,1,237029485,96731508
2300,trum,2016,145,1,134338049,53675220
2197,trum,2022,32,1,88608557,34067578
116,trum,2020,870,1,106215660,44287269
1717,trum,2014,41,2,219788855,87465269
...,...,...,...,...,...,...
2421,truc,2016,1,149008870,666155700,275643201
3070,truc,2013,1,201064925,937686122,391186544
1908,truc,2011,1,221276679,1158490686,487954164
1485,truc,2014,1,230622362,1018634029,417694944


Provider type is on the claim line level. Ideally we should have a provider type for almost all the claims.

In [117]:
pt_sum_df = comp[~comp['provider_type'].isna()].groupby(['data_source', 'year', 'dw_row_count'])['type_count'].sum()
pt_sum_df = pt_sum_df.reset_index()
pt_sum_df['type_to_id'] = 1. * (pt_sum_df['type_count'] / pt_sum_df['dw_row_count'])
pt_sum_df

,data_source,year,dw_row_count,type_count,type_to_id
0,truc,2011,1158490686,1085553957,0.937042
1,truc,2012,1203515059,1129871202,0.938809
2,truc,2013,937686122,920176556,0.981327
3,truc,2014,1018634029,1002905970,0.984560
4,truc,2015,642293491,628117434,0.977929
5,truc,2016,666155700,651388392,0.977832
6,truc,2017,610389935,595380002,0.975409
7,truc,2018,627456941,609973961,0.972137
8,truc,2019,602621228,588920892,0.977265
9,truc,2020,518802742,508463503,0.980071


In [118]:
pt_sum_df[pt_sum_df['type_to_id']  > 5]

,data_source,year,dw_row_count,type_count,type_to_id
